### Grove Motor Controller  

In [ ]:
import meerkat

#### Stop Command for Debugging  
This cell is useful when prototyping.  If a command throws an error in stepper mode before setting both motor speeds to 0, the board will stay in a full duty cycle condition and begin to overheat.

In [ ]:
from motor_grove import GroveMotor

m = GroveMotor(bus_n=1)
m.stop()
print("Stop Done")

#### Test Stepper Motor

In [ ]:
import time

from motor_grove import GroveMotor

m = GroveMotor(bus_n=1)
print("="*20)


# stepper motor microsteps
m.step_micro(1001, 
            delay=0.0001,
            verbose=False)
print("-"*20)
time.sleep(1)
m.step_micro(-1001, 
            delay=0.0001,
            verbose=False)
m.stop()
print("-"*20)

print(m.get("Bench test"))
print(m.writer.device)
"""
# stepper motor full steps
m.step_full(100, 
            delay=0.0001,
            verbose=False)
print("-"*20)
time.sleep(1)
m.step_full(-100, 
            delay=0.0001,
            verbose=False)
m.stop()
print("-"*20)
"""
"""
# dc motor
m.set_frequency(f_Hz=31372)
m.set_direction('cw')
m.set_speed(motor_id=1, motor_speed=100)
time.sleep(4)
m.stop(motor_id=1)
"""

print("="*20)
i = m.get_info()
print(i)